In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.affinity import scale
from shapely.geometry import Point, Polygon
import plotly.express as px
import matplotlib.pyplot as plt
from pandas import DataFrame as DF
from scipy.stats import skew
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from peakutils import baseline
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.cross_decomposition import PLSRegression  # noqa: E402
from sklearn.ensemble import (  # noqa: E402
    ExtraTreesRegressor,  # noqa: E402
)
from scipy.signal import savgol_filter
from pyspectra.transformers.spectral_correction import snv, detrend

import random


#   plot principial components
#  take subset in the PC space of US data


import seaborn as sns

/home/bart2/.pyenv/versions/3.9.5/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [37]:
clay = pd.read_csv("../data/extracted_metadata/clay_334_measured_mir.csv")
df = pd.read_csv("usda_easy.csv")

In [39]:
joined = df.merge(clay, how='inner')
joined = joined.rename(columns={"value_of_measured_property": "clay"})
joined = joined.drop(columns=["soil_property_name", "units_of_measure"])
clay = joined.pop("clay")
joined.insert(4, "clay", clay)
joined


In [45]:
joined

,smp_id,lat_site,long_site,phosphorus,clay,potassium,nitrogen,4001.65608,3999.72758,3997.79907,...,617.12287,615.19436,613.26585,611.33734,609.40883,607.48032,605.55181,603.6233,601.69479,599.76629
0,55487,40.103054,-123.709442,12.837792,25.966192,0.439553,0.397704,0.160430,0.160675,0.160882,...,1.561697,1.558460,1.555115,1.548652,1.537602,1.522870,1.506778,1.491305,1.477490,1.466077
1,55716,40.580002,-88.368889,4.323957,46.963462,0.502973,0.329596,0.248092,0.248465,0.248833,...,1.678972,1.674033,1.667987,1.659683,1.649768,1.639180,1.628478,1.618195,1.608340,1.598872
2,56801,45.287304,-123.009811,20.374913,29.536178,0.803999,0.150141,0.374443,0.375440,0.375685,...,1.687703,1.678620,1.668340,1.657137,1.643722,1.629398,1.616965,1.606175,1.596150,1.587485
3,83647,39.110748,-95.130142,1.887696,46.708213,0.456697,0.120603,0.353162,0.353665,0.354140,...,1.917255,1.913582,1.907787,1.900400,1.891757,1.881545,1.870923,1.861870,1.855070,1.849780
4,83726,39.273613,-101.112053,9.681377,22.088919,1.080621,0.242809,0.171357,0.171783,0.172130,...,1.541585,1.534682,1.527022,1.517158,1.504447,1.490410,1.477635,1.467167,1.458792,1.451975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5018,168237,46.725613,-67.996330,185.644554,16.941367,0.409643,0.188732,0.151832,0.152025,0.152240,...,1.569683,1.570353,1.569655,1.565455,1.557615,1.547972,1.538530,1.530355,1.523492,1.517905
5019,168305,46.143970,-67.901802,313.664546,14.156947,0.391861,0.187639,0.194898,0.195047,0.195183,...,1.590428,1.589757,1.587945,1.582465,1.572915,1.561100,1.548588,1.536510,1.526337,1.518400
5020,168711,37.050083,-122.226501,203.943000,17.777811,1.000051,0.156069,0.251062,0.251545,0.252007,...,1.713895,1.705110,1.695535,1.683763,1.671128,1.658790,1.647680,1.639242,1.632877,1.626618
5021,168856,44.287453,-100.339432,46.381404,26.769655,1.357473,0.275931,0.299812,0.300120,0.300412,...,1.702208,1.697075,1.692110,1.685520,1.676505,1.666918,1.658518,1.651312,1.645137,1.640388


# water


In [43]:
water = pd.read_csv("../data/extracted_metadata/water_unique.csv")


In [49]:
df2 = joined.merge(water)
df2 = df2.rename(columns={"value_of_measured_property": "water"})
df2 = df2.drop(columns=["soil_property_name", "units_of_measure"])
water = df2.pop("water")
df2.insert(4, "water", water)
df2

,smp_id,lat_site,long_site,phosphorus,water,clay,potassium,nitrogen,4001.65608,3999.72758,...,617.12287,615.19436,613.26585,611.33734,609.40883,607.48032,605.55181,603.6233,601.69479,599.76629
0,55487,40.103054,-123.709442,12.837792,12.183544,25.966192,0.439553,0.397704,0.160430,0.160675,...,1.561697,1.558460,1.555115,1.548652,1.537602,1.522870,1.506778,1.491305,1.477490,1.466077
1,55716,40.580002,-88.368889,4.323957,23.463687,46.963462,0.502973,0.329596,0.248092,0.248465,...,1.678972,1.674033,1.667987,1.659683,1.649768,1.639180,1.628478,1.618195,1.608340,1.598872
2,56801,45.287304,-123.009811,20.374913,13.160987,29.536178,0.803999,0.150141,0.374443,0.375440,...,1.687703,1.678620,1.668340,1.657137,1.643722,1.629398,1.616965,1.606175,1.596150,1.587485
3,56801,45.287304,-123.009811,20.374913,13.181818,29.536178,0.803999,0.150141,0.374443,0.375440,...,1.687703,1.678620,1.668340,1.657137,1.643722,1.629398,1.616965,1.606175,1.596150,1.587485
4,83647,39.110748,-95.130142,1.887696,17.815345,46.708213,0.456697,0.120603,0.353162,0.353665,...,1.917255,1.913582,1.907787,1.900400,1.891757,1.881545,1.870923,1.861870,1.855070,1.849780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,168237,46.725613,-67.996330,185.644554,11.710357,16.941367,0.409643,0.188732,0.151832,0.152025,...,1.569683,1.570353,1.569655,1.565455,1.557615,1.547972,1.538530,1.530355,1.523492,1.517905
4357,168305,46.143970,-67.901802,313.664546,8.563672,14.156947,0.391861,0.187639,0.194898,0.195047,...,1.590428,1.589757,1.587945,1.582465,1.572915,1.561100,1.548588,1.536510,1.526337,1.518400
4358,168711,37.050083,-122.226501,203.943000,12.964401,17.777811,1.000051,0.156069,0.251062,0.251545,...,1.713895,1.705110,1.695535,1.683763,1.671128,1.658790,1.647680,1.639242,1.632877,1.626618
4359,168856,44.287453,-100.339432,46.381404,19.162944,26.769655,1.357473,0.275931,0.299812,0.300120,...,1.702208,1.697075,1.692110,1.685520,1.676505,1.666918,1.658518,1.651312,1.645137,1.640388
